In [1]:
import numpy as np
import neo4j

In [2]:
import pandas as pd

In [3]:
driver = neo4j.GraphDatabase.driver("bolt://neo4j:7687", auth=("neo4j", "test"))

In [4]:
session = driver.session()

In [5]:
def run_query(q):
    result = session.run(q)
    return pd.DataFrame([dict(r[0].items()) for r in result])

In [6]:
q = "MATCH (e:Troll) WHERE e.friends_count=1501 RETURN e;"

In [7]:
df = run_query(q)
df.head()

,friends_count,listed_count,favourites_count,verified,created_at,description,time_zone,screen_name,statuses_count,real_statuses_count,followers_count,name,location,id,lang
0,1501,23,1778,false,Fri Aug 16 19:50:47 +0000 2013,•Christian•Constitutional•Capitalist •NRA•Supp...,Eastern Time (US & Canada),EmileeWaren,2170,3192,1909,Emilee Waren,USA,1676481360,en


## Task 2

### List 10 locations that have been mentioned by the largest number of trolls

In [11]:
q = """
MATCH (t:Troll)-[:POSTED]->(:Tweet)-[:CONTAINS_ENTITY]->(l:Location)
RETURN l.name, count(distinct t) as counts
ORDER BY counts DESC
LIMIT 10;"""

In [12]:
result = session.run(q)

In [13]:
df = pd.DataFrame([(r[0], r[1]) for r in result], columns=["Name", "count"])
df

,Name,count
0,Russia,145
1,America,144
2,Florida,103
3,Mexico,100
4,Trump,97
5,Clinton,97
6,Iran,92
7,Syria,91
8,Benghazi,81
9,Texas,81


### List 10 trolls that have been mentioned by the largest number of tweets.

In [101]:
q = """
MATCH (t:Troll)<-[:MENTIONS]-(:Tweet)
RETURN t.screen_name, count(t) as counts
ORDER BY counts DESC
LIMIT 10;"""

In [102]:
result = session.run(q)

In [103]:
df = pd.DataFrame([(r[0], r[1]) for r in result], columns=["Screen name", "count"])
df

,Screen name,count
0,gloed_up,112
1,tpartynews,100
2,rightnpr,16
3,blackmattersus,4
4,brightandglory,3
5,todayinsyria,2
6,dannythehappies,2
7,hyddrox,2
8,holycrapchrix,1
9,heyits_toby,1


### List 10 trolls that have been mentioned by the largest number of users

In [107]:
q = """
MATCH (u:User)-[:POSTED]->(:Tweet)-[:MENTIONS]->(t:Troll)
RETURN t.screen_name, count(DISTINCT u) as counts
ORDER BY counts DESC
LIMIT 10;"""

In [108]:
result = session.run(q)

In [109]:
df = pd.DataFrame([(r[0], r[1]) for r in result], columns=["Name", "count"])
df

,Name,count
0,tpartynews,10
1,blackmattersus,4
2,rightnpr,4
3,gloed_up,4
4,brightandglory,3
5,dannythehappies,2
6,todayinsyria,1
7,holycrapchrix,1
8,heyits_toby,1
9,hyddrox,1


### List the number of tweers per hour (24 results, one per hour), consider the tweets only by users declaring their location in USA.

In [8]:
q = """
MATCH (:User {location: "USA"})-[:POSTED]-(tweet:Tweet)
WHERE size(tweet.created_str)>0
RETURN datetime(replace(tweet.created_str, ' ', 'T')).hour as hour, count(tweet) as counts
ORDER BY counts DESC;"""

In [9]:
result = session.run(q)

In [10]:
df = pd.DataFrame([(r[0], r[1]) for r in result], columns=["Hour", "count"])
df

,Hour,count
0,16,6199
1,15,5901
2,14,5757
3,17,5039
4,12,4936
5,18,4442
6,9,4420
7,8,4412
8,13,4327
9,20,3923


### Find the shortest path between the users: @realdonaldtrump and @hillaryclinton

In [120]:
q = """
MATCH p = shortestPath((d:User)-[*]-(h:User))
WHERE d.screen_name = "realdonaldtrump" AND
      h.screen_name = "hillaryclinton"
RETURN p;"""

In [121]:
result = session.run(q)

In [122]:
a = result.single()

In [131]:
[e for e in a[0]]

[<Relationship id=376658 nodes=(<Node id=2964 labels=frozenset({'Tweet'}) properties={'hashtags': ['Election2016', 'Election2016', 'Election2016', 'Election2016'], 'tweet_id': 776773929151127553, 'expanded_urls': ['http://bit.ly/2cCyMtT', 'http://bit.ly/2cCyMtT', 'http://bit.ly/2cCyMtT', 'http://bit.ly/2cCyMtT'], 'created_at': '1474032311000', 'user_key': 'rh0lbr00k', 'created_str': '2016-09-16 13:25:11', 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'retweeted_status_id': 776765018238922752, 'retweet_count': 0, 'user_id': 1662779209, 'mentions': ['foxnews', 'foxnation', 'realdonaldtrump', 'hillaryclinton'], 'favorite_count': 0, 'text': 'RT @foxnation: . @FoxNews Poll: @HillaryClinton , @realDonaldTrump in One-Point Race Among Likely Voters https://t.co/SiUEfEe4RL #Election2…'}>, <Node id=270161 labels=frozenset({'User'}) properties={'screen_name': 'realdonaldtrump'}>) type='MENTIONS' properties={}>,
 <Relationship id=376659 nodes=(<Node id=2964 labels

### Extend the database with relationship of type :CITES between trolls: we say that a troll :CITES another troll if the former retweeted some tweet(s) by the latter.

In [144]:
q = """
MATCH (t1:Troll)-[:POSTED]->(:Tweet)-[:RETWEETED]->(:Tweet)<-[:POSTED]-(t2:Troll)
WITH DISTINCT t1,  t2
CREATE (t1)-[:CITES]->(t2)
RETURN t1.screen_name, t2.screen_name
ORDER BY t1.screen_name, t2.screen_name"""

In [145]:
result = session.run(q)

In [146]:
df = pd.DataFrame([(r[0], r[1]) for r in result], columns=["Screen name t1", "Screen name t2"])
df

,Screen name t1,Screen name t2
0,Adrienne_GG,ChrixMorgan
1,Adrienne_GG,MelvinSRoberts
2,Aiden7757,DanaGeezus
3,Aiden7757,MelvinSRoberts
4,Aldrich420,GiselleEvns
...,...,...
777,traceyhappymom,queenofthewo
778,traceyhappymom,traceyhappymom
779,wadeharriot,GiselleEvns
780,wadeharriot,TEN_GOP


### Find the 5 most popular hashtags for each of the 10 trolls with the largest number of tweets (return pairs troll screen_name, list of hashtags), use collect() function.

In [136]:
q = """
MATCH (t:Troll)-[:POSTED]->(tw:Tweet)
WITH t, count(tw) as tweets_count ORDER BY tweets_count DESC LIMIT 10
MATCH (t)-[:POSTED]->(tw:Tweet)-[:HAS_TAG]-(tag:Hashtag)
WITH t, tweets_count as ts, tag.tag as tag_name, count(tag) as tags_count ORDER BY tags_count DESC
RETURN t.screen_name, collect(tag_name)[0..5]"""

In [137]:
result = session.run(q)

In [138]:
df = pd.DataFrame([(r[0], r[1]) for r in result], columns=["Screen name", "list of #"])
df

,Screen name,list of #
0,AmelieBaldwin,"[maga, Trump, NeverHillary, Trump2016, TrumpTr..."
1,GiselleEvns,"[MakeMeHateYouInOnePhrase, SometimesItsOkTo, I..."
2,hyddrox,"[maga, Trump, tcot, NeverHillary, Hillary]"
3,PatriotBlake,"[maga, Trump, tcot, NeverHillary, Trump2016]"
4,TheFoundingSon,"[Trump2016, WakeUpAmerica, tcot, LiberalLogic,..."
5,mrclydepratt,"[nowplaying, God, SoundCloud, Listen2, hiphop]"
6,BaoBaeHam,"[nowplaying, Listen2, SoundCloud, God, Tashify]"
7,MelvinSRoberts,"[ItsRiskyTo, SurvivalGuideToThanksgiving, IGet..."
8,brianaregland,"[PodernFamily, nowplaying, HiddenFigures, SciF..."
9,LeroyLovesUSA,"[tcot, maga, pjnet, Trump, IslamKills]"


### For each troll set the value of the new property real_statuses_count as the number of tweets posted by the troll (i.e., the number of Tweets POSTED by the troll in the dump of the database).

In [140]:
q = """
MATCH (t:Troll)-[:POSTED]->(tw:Tweet)
WITH t, count(tw) as counts
SET t += {real_statuses_count: counts}
RETURN t.screen_name, t.real_statuses_count"""

In [141]:
result = session.run(q)

In [142]:
df = pd.DataFrame([(r[0], r[1]) for r in result], columns=["Screen name", "Real statuses count"])
df

,Screen name,Real statuses count
0,SCOTTGOHARD,68
1,Beckster319,56
2,skatewake1994,23
3,KadirovRussia,7
4,MargoSavazh,5
...,...,...
387,Williams8kalvin,3
388,Williams_Diana_,1
389,JoachimBuchwitz,1
390,klara_sauber,829
